In [34]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.exc import OperationalError



In [7]:
file_path = r"D:\data engineering\superMarket project\dataset\supermarket_sales - Sheet1.csv"
df = pd.read_csv(file_path)

print(df.head())


    Invoice ID Branch       City Customer type  Gender  \
0  750-67-8428      A     Yangon        Member  Female   
1  226-31-3081      C  Naypyitaw        Normal  Female   
2  631-41-3108      A     Yangon        Normal    Male   
3  123-19-1176      A     Yangon        Member    Male   
4  373-73-7910      A     Yangon        Normal    Male   

             Product line  Unit price  Quantity   Tax 5%     Total       Date  \
0       Health and beauty       74.69         7  26.1415  548.9715   1/5/2019   
1  Electronic accessories       15.28         5   3.8200   80.2200   3/8/2019   
2      Home and lifestyle       46.33         7  16.2155  340.5255   3/3/2019   
3       Health and beauty       58.22         8  23.2880  489.0480  1/27/2019   
4       Sports and travel       86.31         7  30.2085  634.3785   2/8/2019   

    Time      Payment    cogs  gross margin percentage  gross income  Rating  
0  13:08      Ewallet  522.83                 4.761905       26.1415     9.1  
1  10:

In [75]:
username = 'root' 
password = '1234'  
database = 'super_market' 
host = 'localhost' 

engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}/{database}')

In [76]:
try:
    # Try to establish a connection
    with engine.connect() as connection:
        print("Connection to the database was successful.")

        # Create an inspector object to get metadata
        inspector = inspect(engine)

        # Get all table names in the database
        tables = inspector.get_table_names()
        
        print("Tables in the database:")
        for table in tables:
            print(table)

except OperationalError as e:
    print(f"Could not connect to the database: {e}")

Connection to the database was successful.
Tables in the database:
branch_dimension
customer_dimension
date_dimension
payment_dimension
product_line_dimension
sales_fact


In [52]:

branch_df = df[['Branch', 'City']].drop_duplicates().reset_index(drop=True)

branch_df['branch_id'] = branch_df.index + 1 

branch_df = branch_df.rename(columns={'Branch': 'branch_name', 'City': 'city'})
df = df.rename(columns={'Branch': 'branch_name', 'City': 'city'})

branch_df = branch_df[['branch_id', 'branch_name', 'city']]

branch_df

,branch_id,branch_name,city
0,1,A,Yangon
1,2,C,Naypyitaw
2,3,B,Mandalay


In [38]:
branch_df.to_sql('branch_dimension', con=engine, if_exists='append', index=False)


3

In [54]:
customer_df = df[['Customer type', 'Gender']].drop_duplicates().reset_index(drop=True)
customer_df['customer_id'] = customer_df.index + 1 
customer_df = customer_df[['customer_id', 'Customer type', 'Gender']]
customer_df.rename(columns={'Customer type': 'Customer_type'}, inplace=True)
df.rename(columns={'Customer type': 'Customer_type'}, inplace=True)

customer_df


,customer_id,Customer_type,Gender
0,1,Member,Female
1,2,Member,Male
2,3,Normal,Female
3,4,Normal,Male


In [42]:
customer_df.to_sql('customer_dimension', con=engine, if_exists='append', index=False)


4

In [55]:
product_line_df = df[['Product line']].drop_duplicates().reset_index(drop=True)
product_line_df['product_line_id'] = product_line_df.index + 1 
product_line_df = product_line_df[['product_line_id', 'Product line']]
product_line_df.rename(columns={'Product line': 'Product_line_name'}, inplace=True)
df.rename(columns={'Product line': 'Product_line_name'}, inplace=True)
product_line_df

,product_line_id,Product_line_name
0,1,Health and beauty
1,2,Electronic accessories
2,3,Home and lifestyle
3,4,Sports and travel
4,5,Food and beverages
5,6,Fashion accessories


In [44]:
product_line_df.to_sql('Product_Line_Dimension', con=engine, if_exists='append', index=False)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1440\3357416494.py:1: UserWarning: The provided table name 'Product_Line_Dimension' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  product_line_df.to_sql('Product_Line_Dimension', con=engine, if_exists='append', index=False)


6

In [58]:
payment_df = df[['Payment']].drop_duplicates().reset_index(drop=True)
payment_df['payment_id'] = payment_df.index + 1  
payment_df = payment_df[['payment_id', 'Payment']]
payment_df.rename(columns={'Payment': 'Payment_method'}, inplace=True)
df.rename(columns={'Payment': 'Payment_method'}, inplace=True)
payment_df

,payment_id,Payment_method
0,1,Ewallet
1,2,Cash
2,3,Credit card


In [46]:
payment_df.to_sql('Payment_Dimension', con=engine, if_exists='append', index=False)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1440\3717998022.py:1: UserWarning: The provided table name 'Payment_Dimension' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  payment_df.to_sql('Payment_Dimension', con=engine, if_exists='append', index=False)


3

In [47]:
df['Date'] = pd.to_datetime(df['Date'])
date_df = df[['Date']].drop_duplicates().reset_index(drop=True)
date_df['date_id'] = date_df.index + 1 
date_df['month'] = date_df['Date'].dt.month
date_df['year'] = date_df['Date'].dt.year
date_df['quarter'] = date_df['Date'].dt.quarter
date_df['day_of_week'] = date_df['Date'].dt.dayofweek
date_df = date_df[['date_id', 'Date', 'month', 'year', 'quarter', 'day_of_week']]


In [48]:
date_df.to_sql('Date_Dimension', con=engine, if_exists='append', index=False)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1440\3489084657.py:1: UserWarning: The provided table name 'Date_Dimension' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  date_df.to_sql('Date_Dimension', con=engine, if_exists='append', index=False)


89

In [51]:
print(df.columns)


Index(['Invoice ID', 'Branch', 'City_x', 'Customer type', 'Gender_x',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income',
       'Rating', 'branch_id_x', 'City_y', 'customer_id_x', 'Gender_y',
       'product_line_id_x', 'payment_id_x', 'date_id_x', 'month_x', 'year_x',
       'quarter_x', 'day_of_week_x', 'branch_id_y', 'City', 'customer_id_y',
       'Gender', 'product_line_id_y', 'payment_id_y', 'date_id_y', 'month_y',
       'year_y', 'quarter_y', 'day_of_week_y', 'branch_id', 'City_branch',
       'customer_id', 'Gender_customer', 'product_line_id', 'payment_id',
       'date_id', 'month', 'year', 'quarter', 'day_of_week'],
      dtype='object')


In [59]:
df = df.merge(branch_df, left_on='branch_name', right_on='branch_name', how='left', suffixes=('', '_branch'))
df = df.merge(customer_df, left_on='Customer_type', right_on='Customer_type', how='left', suffixes=('', '_customer'))
df = df.merge(product_line_df, left_on='Product_line_name', right_on='Product_line_name', how='left', suffixes=('', '_product_line'))
df = df.merge(payment_df, left_on='Payment_method', right_on='Payment_method', how='left', suffixes=('', '_payment'))
df = df.merge(date_df, left_on='Date', right_on='Date', how='left', suffixes=('', '_date'))

sales_fact_df = df[['Invoice ID', 'branch_id', 'customer_id', 'product_line_id', 'payment_id', 'date_id', 
                    'Quantity', 'Unit price', 'Tax 5%', 'Total', 'cogs', 'gross income', 'gross margin percentage', 'Rating']]
sales_fact_df.rename(columns={'Invoice ID': 'Invoice_ID'}, inplace=True)
sales_fact_df.rename(columns={'Unit price': 'Unit_price'}, inplace=True)
sales_fact_df.rename(columns={'Tax 5%': 'tax_5_percent'}, inplace=True)
sales_fact_df.rename(columns={'gross income': 'gross_income'}, inplace=True)
sales_fact_df.rename(columns={'gross margin percentage': 'gross_margin_percentage'}, inplace=True)
df.rename(columns={'Customer type': 'Customer_type'}, inplace=True)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1440\2626618897.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_fact_df.rename(columns={'Invoice ID': 'Invoice_ID'}, inplace=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1440\2626618897.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_fact_df.rename(columns={'Unit price': 'Unit_price'}, inplace=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1440\2626618897.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [68]:
print(sales_fact_df.columns)


Index(['Invoice_ID', 'branch_id', 'customer_id', 'product_line_id',
       'payment_id', 'date_id', 'Quantity', 'Unit_price', 'tax_5_percent',
       'Total', 'cogs', 'gross_income', 'gross_margin_percentage', 'Rating'],
      dtype='object')


In [69]:
sales_fact_df['Invoice_ID'] = sales_fact_df['Invoice_ID'].astype(str) 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1440\3620548426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_fact_df['Invoice_ID'] = sales_fact_df['Invoice_ID'].astype(str)


In [73]:
sales_fact_df = sales_fact_df.drop_duplicates(subset='Invoice_ID', keep='first') 


In [77]:
sales_fact_df.to_sql('Sales_Fact', con=engine, if_exists='append', index=False)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1440\2406274448.py:1: UserWarning: The provided table name 'Sales_Fact' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  sales_fact_df.to_sql('Sales_Fact', con=engine, if_exists='append', index=False)


1000